In [1]:
import  psycopg2 
import config as creds

def connect():    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.pg_rds_endpoint +" port="+ "5432" +" dbname="+ creds.pg_db_name +" user=" + creds.pg_user \
                  +" password="+ creds.pg_pwd
    
    conn = psycopg2.connect(conn_string)
    print(conn)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

def disconnect(conn, cursor):
    cursor.close()
    conn.close()
      
def load_data_onto_db(cursor):
    with open('./resources/fifa_world_players_clean.csv', 'r', encoding="UTF8") as row:
        next(row) # Skip the header row.
        cursor.copy_from(row, 'players', sep=',')


In [2]:
# Purpose:
# To house common functions

def create_fifa_players_dataframe(dataset_location):
    import pandas

    return pandas.read_csv(dataset_location, encoding='UTF-8')

def drop_columns_from_dataframe(data, columns):
    data_copy = data.copy()

    for column in data.columns:
        if column not in columns:
            del data[column]

    return data

def create_players_dataframes(data, columns_to_keep):
    
    filtered_data = drop_columns_from_dataframe(data, columns_to_keep)
        
    return filtered_data

def create_schema_off_dataset(data, file_name):
    import json
    import pandas
    schema = pandas.io.json.build_table_schema(data, index=False, primary_key=None, version=True)
    # print(schema)
    with open(file_name, 'w') as file:
        file.write(json.dumps(schema))
    
def encode_field(data, column):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    data_copy = data.copy()
    data_copy[column] = le.fit_transform(data[column])
    # display(data_copy)
    return data_copy

columns_to_keep = ["overall", "potential", "field_position", "height","weight","crossing","finishing","heading_accuracy","short_passing","volleys",
"dribbling","curve","fk_accuracy","long_passing","ball_control","acceleration","sprint_speed","agility",
"reactions","balance","shot_power","jumping","stamina","strength","long_shots","aggression","interceptions",
"positioning","vision","penalties","composure","marking","standing_tackle","sliding_tackle","gk_diving",
"gk_handling","gk_kicking","gk_positioning","gk_reflexes", "rating"]

cols_to_drop_for_keepers = ['photo', 'flag',
    'international_reputation', 'Weak Foot', 'Skill Moves', 'Work Rate'
     'Body Type', 'Real Face',  'Jersey Number',
    'Joined', 'Loaned From', 'Contract Valid Until',
     'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys',
       'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl',
       'Acceleration', 'SprintSpeed',
     'Club Logo', 'Release Clause', 'Special', 'Preferred Foot',
     'Marking', 'StandingTackle', 'SlidingTackle']

cols_to_drop_for_players = ['Photo', 'Flag',
    'International Reputation', 'Weak Foot', 'Jersey Number', 'Work Rate'
    'Joined', 'Loaned From', 'Contract Valid Until', 'Body Type', 'Real Face', 'Skill Moves', 
     'Photo', 'Flag', 'Club Logo', 'Release Clause', 'Special', 'Preferred Foot', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']

def create_fifa_players_dataset(data_in, file_loc, csv_file):
    from pathlib import Path
    
    # check and drop nulls
    count_na = data_in.isna().sum().sum()
   
    if count_na > 0:
        data_in = data_in.dropna()

    # source: https://stackoverflow.com/questions/48366506/calculate-new-column-as-the-mean-of-other-columns-pandas/48366525
    col = data_in[["overall","potential"]]
    data_in['rating'] = col.mean(axis=1)
    
    data_in = data_in.infer_objects()
    # display(data_in.head)

    %time data_in.to_csv(Path(file_loc, csv_file), index = False, line_terminator='\n')

    # if show:
    #     print(col.mean(axis=1))
    #     print(data_in.info())
    #     print(data_in.describe(exclude =[object, int]))

    return data_in

In [3]:
def adjust_rating(x):
    if  x > 85:
        return "Excellent"
    # elif 70 < x <= 85:
    #     return "Good"
    return "Decent"

In [4]:
														
goal_keepers = ['GK']
defenders = ['RWB', 'RCB', 'RB', 'LCB', 'CB', 'LWB', 'LB']
def_mid_fielders = ['LDM', 'CDM','RDM']
mid_fielders = ['CM', 'RCM', 'RM', 'LCM', 'LM']
attack_mid_fielders = ['LAM', 'CAM', 'RAM']
forwards = ['RF', 'CF','LF', 'LW', 'RW']
strikers = ['ST', 'RS', 'LS'] # Striker or Center Forward